Code attribution:



*   https://www.analyticsvidhya.com/blog/2021/09/performing-email-spam-detection-using-bert-in-python/
*   https://github.com/prateekjoshi565/Fine-Tuning-BERT/blob/master/Fine_Tuning_BERT_for_Spam_Classification.ipynb



In [2]:
!pip install -U "tensorflow==2.8.*"
!pip install -U "tensorflow-text==2.8.*"
!pip install transformers
!pip install -U tensorflow-text
!pip install transformers[torch]
!pip install accelerate -U

  Using cached tensorflow-2.8.4-cp310-cp310-manylinux2010_x86_64.whl (498.1 MB)
  Using cached protobuf-3.19.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.1 MB)
  Using cached tensorboard-2.8.0-py3-none-any.whl (5.8 MB)
  Using cached tensorflow_estimator-2.8.0-py2.py3-none-any.whl (462 kB)
  Using cached keras-2.8.0-py2.py3-none-any.whl (1.4 MB)
  Using cached google_auth_oauthlib-0.4.6-py2.py3-none-any.whl (18 kB)
  Using cached tensorboard_data_server-0.6.1-py3-none-manylinux2010_x86_64.whl (4.9 MB)
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.13.0
    Uninstalling tensorflow-estimator-2.13.0:
      Successfully uninstalled tensorflow-estimator-2.13.0
  Attempting uninstall: keras
    Found existing installation: keras 2.13.1
    Uninstalling keras-2.13.1:
      Successfully uninstalled keras-2.13.1
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.1
 

  Using cached tensorflow_text-2.8.2-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (4.9 MB)
  Attempting uninstall: tensorflow-text
    Found existing installation: tensorflow-text 2.13.0
    Uninstalling tensorflow-text-2.13.0:
      Successfully uninstalled tensorflow-text-2.13.0


  Using cached tensorflow_text-2.13.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.5 MB)
  Using cached tensorflow-2.13.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (524.1 MB)
  Using cached keras-2.13.1-py3-none-any.whl (1.7 MB)
  Using cached protobuf-4.24.0-cp37-abi3-manylinux2014_x86_64.whl (311 kB)
  Using cached tensorboard-2.13.0-py3-none-any.whl (5.6 MB)
  Using cached tensorflow_estimator-2.13.0-py2.py3-none-any.whl (440 kB)
  Using cached google_auth_oauthlib-1.0.0-py2.py3-none-any.whl (18 kB)
  Using cached tensorboard_data_server-0.7.1-py3-none-manylinux2014_x86_64.whl (6.6 MB)
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.6.1
    Uninstalling tensorboard-data-server-0.6.1:
   

In [1]:
import tensorflow_hub as hub
import pandas as pd
import tensorflow_text as text
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
import transformers
from transformers import AutoModel, BertTokenizerFast, AutoModelForSequenceClassification, TrainingArguments
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# specify GPU
device = torch.device("cuda")

In [3]:
# load data
df = pd.read_csv('/content/Data/spam.csv')
df.head()

,Category,Message
0,not phishing,"Go until jurong point, crazy.. Available only ..."
1,not phishing,Ok lar... Joking wif u oni...
2,phishing,Free entry in 2 a wkly comp to win FA Cup fina...
3,not phishing,U dun say so early hor... U c already then say...
4,not phishing,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
# check count and unique and top values and their frequency
df['Category'].value_counts()


not phishing    4825
phishing         747
Name: Category, dtype: int64

In [6]:
# check percentage of data - states how much data needs to be balanced
print(str(round(747/4825,2))+'%')

0.15%


In [7]:
# creating 2 new dataframe as df_phishing, df_not_phishing

df_phishing = df[df['Category']=='phishing']

df_not_phishing = df[df['Category']=='not phishing']

print("Not Phishing Dataset Shape:", df_not_phishing.shape)

print("Phishing Dataset Shape:", df_phishing.shape)

Not Phishing Dataset Shape: (4825, 2)
Phishing Dataset Shape: (747, 2)


In [8]:
# downsampling not phishing dataset - take only random 747 example
# will use df_phishing.shape[0] - 747
df_nphishing_downsampled = df_not_phishing.sample(df_phishing.shape[0])
df_nphishing_downsampled.shape

(747, 2)

In [9]:
# concating both dataset - df_phishing and df_nphishing_downsampled to create df_balanced dataset
df_balanced = pd.concat([df_phishing , df_nphishing_downsampled])

df_balanced['Category'].value_counts()

phishing        747
not phishing    747
Name: Category, dtype: int64

In [10]:
df_balanced.sample(10)

,Category,Message
3532,not phishing,Prepare to be pounded every night...
5547,phishing,Had your contract mobile 11 Mnths? Latest Moto...
4703,not phishing,Anytime...
1573,phishing,Ur cash-balance is currently 500 pounds - to m...
4944,not phishing,Check mail.i have mailed varma and kept copy t...
4500,phishing,"SMS SERVICES. for your inclusive text credits,..."
2689,not phishing,Yes I know the cheesy songs from frosty the sn...
685,not phishing,I wanted to ask ü to wait 4 me to finish lect....
2267,phishing,<Forwarded from 88877>FREE entry into our £250...
1396,not phishing,Thats cool! I am a gentleman and will treat yo...


In [11]:
# creating numerical repersentation of category - one hot encoding
df_balanced['Type'] = df_balanced['Category'].apply(lambda x:1 if x=='phishing' else 0)

# displaying data - phishing -1 , not phishing-0
df_balanced.sample(4)

,Category,Message,Type
3778,phishing,"Mila, age23, blonde, new in UK. I look sex wit...",1
3820,not phishing,"sorry brah, just finished the last of my exams...",0
962,phishing,URGENT! We are trying to contact U. Todays dra...,1
5568,not phishing,Will ü b going to esplanade fr home?,0


In [12]:
# loading train test split
from sklearn.model_selection import train_test_split
X_train, X_test , y_train, y_test = train_test_split(df_balanced['Message'], df_balanced['Type'],
                                                    stratify = df_balanced['Type'])

In [13]:
# downloading preprocessing files and model
bert_preprocessor = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
bert_encoder = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4')


In [14]:
# Training using the Keras API
text_input = tf.keras.layers.Input(shape = (), dtype = tf.string, name = 'Inputs')
preprocessed_text = bert_preprocessor(text_input)
embeed = bert_encoder(preprocessed_text)
dropout = tf.keras.layers.Dropout(0.1, name = 'Dropout')(embeed['pooled_output'])
outputs = tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'Dense')(dropout)
# creating final model
model = tf.keras.Model(inputs = [text_input], outputs = [outputs])

In [15]:
# check the summary of the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 Inputs (InputLayer)         [(None,)]                    0         []                            
                                                                                                  
 keras_layer (KerasLayer)    {'input_type_ids': (None,    0         ['Inputs[0][0]']              
                             128),                                                                
                              'input_mask': (None, 128)                                           
                             , 'input_word_ids': (None,                                           
                              128)}                                                               
                                                                                              

In [16]:
Metrics = [tf.keras.metrics.BinaryAccuracy(name = 'accuracy'),
           tf.keras.metrics.Precision(name = 'precision'),
           tf.keras.metrics.Recall(name = 'recall')
           ]
# compiling our model
model.compile(optimizer ='adam',
               loss = 'binary_crossentropy',
               metrics = Metrics)

In [17]:
history = model.fit(X_train, y_train, epochs = 10)

Epoch 1/10
35/35 [==============================] - 652s 18s/step - loss: 0.6331 - accuracy: 0.6277 - precision: 0.6307 - recall: 0.6161
Epoch 2/10
35/35 [==============================] - 624s 18s/step - loss: 0.4955 - accuracy: 0.8268 - precision: 0.8000 - recall: 0.8714
Epoch 3/10
35/35 [==============================] - 615s 18s/step - loss: 0.4255 - accuracy: 0.8786 - precision: 0.8681 - recall: 0.8929
Epoch 4/10
35/35 [==============================] - 616s 18s/step - loss: 0.3804 - accuracy: 0.8732 - precision: 0.8732 - recall: 0.8732
Epoch 5/10
35/35 [==============================] - 625s 18s/step - loss: 0.3524 - accuracy: 0.8821 - precision: 0.8728 - recall: 0.8946
Epoch 6/10
35/35 [==============================] - 616s 18s/step - loss: 0.3263 - accuracy: 0.8982 - precision: 0.8940 - recall: 0.9036
Epoch 7/10
35/35 [==============================] - 620s 18s/step - loss: 0.3078 - accuracy: 0.9062 - precision: 0.8957 - recall: 0.9196
Epoch 8/10
35/35 [=======================

In [18]:
# Evaluating performance
model.evaluate(X_test,y_test)

12/12 [==============================] - 227s 19s/step - loss: 0.2834 - accuracy: 0.9118 - precision: 0.9053 - recall: 0.9198


[0.2834291160106659,
 0.9117646813392639,
 0.9052631855010986,
 0.9197860956192017]

1/1 [==============================] - 5s 5s/step
[['phishing']
 ['phishing']
 ['phishing']
 ['not phishing']
 ['phishing']
 ['not phishing']]
